Настройка окружения в Google Colab

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

tensorflow и keras - основные библиотеки для нейронных сетей

numpy - для работы с числовыми массивами

matplotlib - для визуализации

Загрузка и подготовка данных IMDB

In [2]:
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Загружаем встроенный набор данных IMDB

num_words=10000 - сохраняем только 10000 самых частых слов

Данные уже разделены на обучающие и тестовые

train_data - списки индексов слов (отзывы)

train_labels - 0 (негативный) или 1 (позитивный)

In [ ]:
# Загрузка встроенного набора данных
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

# Преобразование целочисленных последовательностей в бинарные матрицы
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

# Векторизация меток
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

Подготовка данных

In [3]:
word_index = imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Получаем словарь соответствия слов и индексов

Добавляем специальные токены:

<PAD> - для заполнения до одинаковой длины

<START> - начало последовательности

<UNK> - неизвестные слова

<UNUSED> - неиспользуемые слова

In [4]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                      value=word_index["<PAD>"],
                                                      padding='post',
                                                      maxlen=256)
test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                     value=word_index["<PAD>"],
                                                     padding='post',
                                                     maxlen=256)

Приводим все отзывы к одинаковой длине (256 слов)

padding='post' - заполняем нулями в конце

value=word_index["<PAD>"] - используем индекс заполнителя

Создание модели

In [5]:
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

Embedding - слой преобразует индексы слов в плотные векторы (16-мерные)

GlobalAveragePooling1D - усредняет векторы слов, получая вектор для всего отзыва

Dense(16) - полносвязный слой с 16 нейронами и ReLU активацией

Dense(1) - выходной слой с сигмоидой (вероятность позитивного отзыва)

Компиляция модели

In [6]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

adam - адаптивный оптимизатор

binary_crossentropy - функция потерь для бинарной классификации

accuracy - метрика (доля правильных ответов)

Обучение модели

In [7]:
history = model.fit(train_data,
                    train_labels,
                    epochs=40,
                    batch_size=512,
                    validation_split=0.2,
                    verbose=1)

Epoch 1/40
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.5436 - loss: 0.6910 - val_accuracy: 0.5788 - val_loss: 0.6797
Epoch 2/40
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6767 - loss: 0.6732 - val_accuracy: 0.7312 - val_loss: 0.6469
Epoch 3/40
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7229 - loss: 0.6356 - val_accuracy: 0.7480 - val_loss: 0.5971
Epoch 4/40
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7741 - loss: 0.5833 - val_accuracy: 0.8016 - val_loss: 0.5369
Epoch 5/40
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.8235 - loss: 0.5171 - val_accuracy: 0.8292 - val_loss: 0.4786
Epoch 6/40
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.8397 - loss: 0.4562 - val_accuracy: 0.8428 - val_loss: 0.4318
Epoch 7/40
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8537 - loss: 0.4079 - val_accuracy: 0.8530 - val_loss: 0.3958
Epoch 8/40
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8671 - loss: 0.3674 - val_accuracy: 0.8578 - v

epochs=40 - количество эпох обучения

batch_size=512 - размер пакета

validation_split=0.2 - 20% данных для валидации

verbose=1 - вывод прогресса обучения

Оценка модели

In [8]:
results = model.evaluate(test_data, test_labels, verbose=2)
print(f"Test loss: {results[0]}, Test accuracy: {results[1]}")

782/782 - 1s - 2ms/step - accuracy: 0.8658 - loss: 0.3386
Test loss: 0.33855733275413513, Test accuracy: 0.8657600283622742


# Многоклассовая классификация вопросов Stack Overflow

Загрузка и подготовка данных

In [15]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz
!tar -xvf stack_overflow_16k.tar.gz

Выходные данные были обрезаны до нескольких последних строк (5000).
test/csharp/1993.txt
test/csharp/1039.txt
test/csharp/1987.txt
test/csharp/220.txt
test/csharp/546.txt
test/csharp/552.txt
test/csharp/234.txt
test/csharp/1978.txt
test/csharp/1788.txt
test/csharp/1950.txt
test/csharp/208.txt
test/csharp/1944.txt
test/csharp/787.txt
test/csharp/1213.txt
test/csharp/1575.txt
test/csharp/1561.txt
test/csharp/1207.txt
test/csharp/793.txt
test/csharp/963.txt
test/csharp/1549.txt
test/csharp/977.txt
test/csharp/744.txt
test/csharp/750.txt
test/csharp/988.txt
test/csharp/778.txt
test/csharp/1367.txt
test/csharp/195.txt
test/csharp/1401.txt
test/csharp/1415.txt
test/csharp/181.txt
test/csharp/1373.txt
test/csharp/57.txt
test/csharp/817.txt
test/csharp/5.txt
test/csharp/1429.txt
test/csharp/43.txt
test/csharp/803.txt
test/csharp/630.txt
test/csharp/156.txt
test/csharp/142.txt
test/csharp/624.txt
test/csharp/94.txt
test/csharp/1398.txt
test/csharp/80.txt
test/csharp/618.txt
test/csharp/1603.txt

In [16]:
base_dir = './stack_overflow_16k'

In [11]:
import os

In [17]:
# Импорт необходимых библиотек
import tensorflow as tf
from tensorflow import keras
import os
import matplotlib.pyplot as plt

# Загрузка данных Stack Overflow
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"
dataset_dir = keras.utils.get_file(
    "stack_overflow_16k.tar.gz",
    dataset_url,
    extract=True
)

# Правильный путь к данным (распакованный архив создает папку 'stack_overflow_16k')
base_dir = os.path.join(os.path.dirname(dataset_dir), 'stack_overflow_16k')

# Проверяем содержимое папки
print("Содержимое директории:", os.listdir(base_dir))

# Параметры загрузки данных
batch_size = 32
seed = 42

# Создание тренировочного, валидационного и тестового наборов данных
raw_train_ds = keras.utils.text_dataset_from_directory(
    os.path.join(base_dir, 'train'),  # Путь к тренировочным данным
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

raw_val_ds = keras.utils.text_dataset_from_directory(
    os.path.join(base_dir, 'train'),  # Те же данные, но для валидации
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

raw_test_ds = keras.utils.text_dataset_from_directory(
    os.path.join(base_dir, 'test'),  # Тестовые данные
    batch_size=batch_size)


FileNotFoundError: [Errno 2] No such file or directory: '/root/.keras/datasets/stack_overflow_16k'

Создаем датасеты из текстовых файлов

Разделяем на обучающую (80%), валидационную (20%) и тестовую выборки

batch_size=32 - размер пакета

seed=42 - фиксируем случайное seed для воспроизводимости


Векторизация текста

In [ ]:
max_features = 10000
sequence_length = 250

vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

Создаем слой для векторизации текста

max_features=10000 - максимальное количество слов

sequence_length=250 - максимальная длина текста

output_mode='int' - преобразуем слова в целочисленные индексы

In [ ]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

Адаптируем слой векторизации к обучающим данным

Создание модели

In [ ]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(max_features + 1, 64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4)])

vectorize_layer - наш слой векторизации

Embedding - слой векторных представлений слов (64-мерные)

Dropout - слои для регуляризации (предотвращение переобучения)

GlobalAveragePooling1D - усреднение по последовательности

Dense(4) - выходной слой (4 класса тегов)

Компиляция и обучение

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(
    raw_train_ds,
    validation_data=raw_val_ds,
    epochs=10)

SparseCategoricalCrossentropy - функция потерь для многоклассовой классификации

adam - оптимизатор

accuracy - метрика

Обучаем 10 эпох

 Оценка модели

In [ ]:
loss, accuracy = model.evaluate(raw_test_ds)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

Вычисляем потери и точность на тестовых данных

Выводим результаты

Предсказание на новых данных

In [ ]:
export_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Activation('softmax')
])

predictions = export_model.predict(["how to parse json in python"])
print(predictions)

Добавляем softmax для получения вероятностей классов

Делаем предсказание для нового вопроса

Выводим вероятности для каждого класса (тега)